# investigate-SMTG-data
2.10.23

What does the AD SMTG DIA data look like from the MacCoss lab? 
Are there any technical replicates? Could I potentially repeat my distributions 
experiment on this dataset? 

Also writing the pre-processed SMTG peptide quants matrix (PXD034525) to a csv, to 
be stuck in the `data/peptides_data/` directory. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# plotting templates
sns.set(context="talk", style="ticks") 
sns.set_palette("tab10")

#### Configs

In [2]:
smtg_path = "../../../../data/maccoss-data/SMTG-pepGrp.batchadj.csv"
n_reps = 5
pxd = "PXD034525"

#### Helper functions 

In [3]:
def get_mean_var_std(df, n_replicates):
    """ 
    For the intensity cols of a single MaxQuant dataframe, break down
    column-wise according to the number of replicates in the matrix, 
    then get means and variances for each peptide, for each replicate.
    Also get peptide standard deviations. 
    
    Assumes columns are in sorted order! as in, the replicate runs are
    next to one another. 
    
    Parameters
    ----------
    df: Matrix of intensity vals, for a single PRIDE experiment
    n_replicates: Number of replicates for each run within that PRIDE
                    experiment
    
    Returns
    -------
    all_means_flat: Flattened (1D) array of all peptide means
    all_vars_flat: Flattened (1D) array of all peptide variances
    all_stds_flat: Flattened (1D) array of all peptide standard deviations
    """
    
    cols = list(df.columns)
    r_split_cols = [cols[i:i+n_replicates] for i in range(0,len(cols), n_replicates)]

    all_means = []
    all_vars = []
    all_stds = []
    
    for i in df.index:
        peptide_means = []
        peptide_vars = []
        peptide_stds = []
    
        for j in range(0,len(r_split_cols)):
            rep_df = df[r_split_cols[j]]
            peptide_mean = np.nanmean(rep_df.iloc[i])
            peptide_var = np.nanvar(rep_df.iloc[i])
            peptide_std = np.nanstd(rep_df.iloc[i])
        
            peptide_means.append(peptide_mean)
            peptide_vars.append(peptide_var)
            peptide_stds.append(peptide_std)
            
        all_means.append(peptide_means)
        all_vars.append(peptide_vars)
        all_stds.append(peptide_stds)
        
    all_means_flat = np.array(all_means).flatten()
    all_vars_flat = np.array(all_vars).flatten()
    all_stds_flat = np.array(all_stds).flatten()
        
    return all_means_flat, all_vars_flat, all_stds_flat

#### Read in and pre-process
The idea is that each "HADT" sample is a replicate, as is each "TRPR" sample. 

In [4]:
# read in 
smtg_peps = pd.read_csv(smtg_path)

keep_cols = ['HADT01', 'TRPR01', 'HADT02', 'TRPR02', 'HADT03', 'TRPR03', 'HADT04',
            'TRPR04', 'HADTP05', 'TRPR05']

# subset
smtg_peps = smtg_peps[keep_cols]

# reorder columns so that replicates are next to each other
smtg_peps = smtg_peps[
    ["HADT01", "HADT02", "HADT03", "HADT04", "HADTP05", 
     "TRPR01", "TRPR02", "TRPR03", "TRPR04", "TRPR05"]
]

# set zeros to nans...
    # though Mike says there may be a difference between 0s and NaNs in this context
smtg_peps[smtg_peps == 0] = np.nan

# convert to np array
quants_mat = np.array(smtg_peps)
quants_mat

# inverse log transform - 2^x, for every element in quants_mat
    # and also smtg_peps, the pandas dataframe version 
quants_mat = np.power(2, quants_mat)
smtg_peps = np.power(2, smtg_peps)

# write to csv
smtg_peps.to_csv("PXD034525_peptides.csv", index=False)

#### Get the means and vars across technical replicates

In [ ]:
# init the aggregated dataframe
mean_x_var = pd.DataFrame(columns=["means", "variances", "stds", "PXD"])

# get the means and vars
means, variances, stds = get_mean_var_std(smtg_peps, n_reps)

# add to the aggregated dataframe
mean_x_var["means"] = means
mean_x_var["variances"] = variances
mean_x_var["stds"] = stds
mean_x_var["PXD"] = pxd

# get rid of zero means and vars
mean_x_var = mean_x_var[mean_x_var["variances"] != 0]
mean_x_var = mean_x_var[mean_x_var["means"] != 0]

# get rid of nan entries
nan_idxs = np.isnan(mean_x_var["means"]) | np.isnan(mean_x_var["variances"])
mean_x_var = mean_x_var[~nan_idxs]

mean_x_var = mean_x_var.reset_index(drop=True)

/net/gs/vol3/software/modules-sw-python/3.7.7/jupyter/5.7.9/Linux/CentOS7/x86_64/ipykernel_launcher.py:38: RuntimeWarning: Mean of empty slice
/net/gs/vol3/software/modules-sw-python/3.7.7/jupyter/5.7.9/Linux/CentOS7/x86_64/ipykernel_launcher.py:39: RuntimeWarning: Degrees of freedom <= 0 for slice.
/net/noble/vol2/home/lincolnh/.local/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1671: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


#### Plot

In [ ]:
plt.figure(figsize=(6,6))
sns.scatterplot(
    data=mean_x_var, x="means", y="variances", alpha=0.05, 
    edgecolors="none", label="SMTG Peptides")

plt.minorticks_off()

plt.xlabel("Mean")
plt.ylabel("Variance")

plt.xscale("log")
plt.yscale("log")

ax = plt.gca()

lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
        ]

ax.plot(lims, lims, color="black", zorder=0, alpha=0.7, label='y=x')
ax.set_aspect('equal', 'box')

plt.legend(
    frameon=True, 
    fancybox=False, 
    loc="upper left", 
    edgecolor="black",
    handlelength=0.5,
    borderaxespad=0,
    fontsize="small",
)

plt.tight_layout()
plt.show()
plt.close()

***

## Logged version 

In [ ]:
# log the peptide quants
smtg_peps = np.log(smtg_peps)

# init the aggregated dataframe
mean_x_var_log = pd.DataFrame(columns=["means", "variances", "stds", "PXD"])

# get the means and vars
means, variances, stds = get_mean_var_std(smtg_peps, n_reps)

# add to the aggregated dataframe
mean_x_var_log["means"] = means
mean_x_var_log["variances"] = variances
mean_x_var_log["stds"] = stds
mean_x_var_log["PXD"] = pxd

# get rid of zero means and vars
mean_x_var_log = mean_x_var_log[mean_x_var_log["variances"] != 0]
mean_x_var_log = mean_x_var_log[mean_x_var_log["means"] != 0]

# get rid of nan entries
nan_idxs = np.isnan(mean_x_var_log["means"]) | np.isnan(mean_x_var_log["variances"])
mean_x_var_log = mean_x_var_log[~nan_idxs]

mean_x_var_log = mean_x_var_log.reset_index(drop=True)

#### Plot the logged quants

In [ ]:
plt.figure(figsize=(6,6))
sns.scatterplot(
    data=mean_x_var_log, x="means", y="variances", alpha=0.05, 
    edgecolors="none", label="SMTG Peptides")

plt.minorticks_off()

plt.xlabel("Mean")
plt.ylabel("Variance")

#plt.xscale("log")
#plt.yscale("log")

ax = plt.gca()

lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
        ]

ax.plot(lims, lims, color="black", zorder=0, alpha=0.7, label='y=x')
ax.set_aspect('equal', 'box')

plt.legend(
    frameon=True, 
    fancybox=False, 
    loc="upper left", 
    edgecolor="black",
    handlelength=0.5,
    borderaxespad=0,
    fontsize="small",
)

plt.tight_layout()
plt.show()
plt.close()